<h1>Federated Learning - GTEx_V8 Example</h1>
<h2>Populate remote PyGrid nodes with labeled tensors </h2>
In this notebook, we will populate our PyGrid nodes with labeled data so that it will be used later by people interested in train models.

**NOTE:** At the time of running this notebook, we were running the grid components in background mode.  

Components:
 - PyGrid Network (http://localhost:5000)
 - PyGrid Node h1 (http://localhost:3000)
 - PyGrid Node h2 (http://localhost:3001)
 
Code implementation for this notebook has been referred from <a href="https://github.com/OpenMined/PySyft/blob/master/examples/tutorials/grid/federated_learning/mnist/Fed.Learning%20MNIST%20%5B%20Part-1%20%5D%20-%20Populate%20a%20Grid%20Network%20(%20Dataset%20).ipynb">Fed.Learning MNIST [ Part-1 ] - Populate a Grid Network ( Dataset )</a> tutorial

<h2>Import dependencies</h2>

In [5]:
#dependencies for helper functions/classes
import pandas as pd
import pyarrow.parquet as pq
from typing import NamedTuple
import os.path as path
import os
import progressbar
import requests
import numpy as np
import random


#keras for ML
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dropout, Input, Dense
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.utils import plot_model, normalize
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import SGD, Adam, Nadam, Adadelta
from tensorflow.keras.activations import relu, elu, sigmoid

#sklearn for preprocessing the data and train-test split
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, accuracy_score, classification_report
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, r2_score, mean_squared_error, mean_absolute_error

#for plots
import matplotlib
import matplotlib.pyplot as plt

#%matplotlib inline

### # Parameter cell -->

In [6]:
seed = 7
dataset_size = 17000
n_classes = 6
no_samples_to_take = dataset_size // n_classes

# Connect directly to grid nodes
nodes = ["ws://0.0.0.0:3000/",
         "ws://0.0.0.0:3001/"]

In [7]:
import syft as sy

#########<syft==0.2.8>#######################
# # Dynamic FL -->
from syft.grid.clients.data_centric_fl_client import DataCentricFLClient

# #Static FL -->
from syft.grid.clients.model_centric_fl_client import ModelCentricFLClient
#############################################

import torch
import pickle
import time
import numpy as np
import torchvision
from torchvision import datasets, transforms

In [8]:
class Labels(NamedTuple):
    '''
    One-hot labeled data
    '''
    tissue: np.ndarray
    sex: np.ndarray
    age: np.ndarray
    death: np.ndarray
        

class Genes:
    '''
    Class to load GTEX samples and gene expressions data
    '''
    def __init__(self, samples_path: str = '', expressions_path: str = '', problem_type: str = "classification"):
        self.__set_samples(samples_path)
        self.__set_labels(problem_type)
        if expressions_path != '':
            self.expressions = self.get_expressions(expressions_path)

    def __set_samples(self, sample_path: str) -> pd.DataFrame:
        self.samples: pd.DataFrame = pq.read_table(sample_path).to_pandas()
        self.samples["Death"].fillna(-1.0, inplace = True)
        self.samples: pd.DataFrame = self.samples.set_index("Name")
        self.samples["Sex"].replace([1, 2], ['male', 'female'], inplace=True)
        self.samples["Death"].replace([-1,0,1,2,3,4], ['alive/NA', 'ventilator case', '<10 min.', '<1 hr', '1-24 hr.', '>1 day'], inplace=True)
        self.samples = self.samples[~self.samples['Death'].isin(['>1 day'])]
        return self.samples

    def __set_labels(self, problem_type: str = "classification") -> Labels:
        self.labels_list = ["Tissue", "Sex", "Age", "Death"]
        self.labels: pd.DataFrame = self.samples[self.labels_list]
        self.drop_list = self.labels_list + ["Subtissue", "Avg_age"]
        
        if problem_type == "classification":
            dummies_df = pd.get_dummies(self.labels["Age"])
            print(dummies_df.columns.tolist())
            self.Y = dummies_df.values
        
        if problem_type == "regression":
            self.Y = self.samples["Avg_age"].values
        
        return self.Y

    def sex_output(self, model):
        return Dense(units=self.Y.sex.shape[1], activation='softmax', name='sex_output')(model)

    def tissue_output(self, model):
        return Dense(units=self.Y.tissue.shape[1], activation='softmax', name='tissue_output')(model)

    def death_output(self, model):
        return Dense(units=self.Y.death.shape[1], activation='softmax', name='death_output')(model)

    def age_output(self, model):
        '''
        Created an output layer for the keras mode
        :param model: keras model
        :return: keras Dense layer
        '''
        return Dense(units=self.Y.age.shape[1], activation='softmax', name='age_output')(model)


    def get_expressions(self, expressions_path: str)->pd.DataFrame:
        '''
        load gene expressions DataFrame
        :param expressions_path: path to file with expressions
        :return: pandas dataframe with expression
        
        '''
        
        if expressions_path.endswith(".parquet"):
            return pq.read_table(expressions_path).to_pandas().set_index("Name") 
        else:
            separator = "," if expressions_path.endswith(".csv") else "\t"
            return pd.read_csv(expressions_path, sep=separator).set_index("Name") 

    def prepare_data(self, normalize_expressions: bool = True)-> np.ndarray:
        '''
        :param normalize_expressions: if keras should normalize gene expressions
        :return: X array to be used as input data by keras
        '''
        data = self.samples.join(self.expressions, on = "Name", how="inner")
        ji = data.columns.drop(self.drop_list)
        x = data[ji]
        
        # adding one-hot-encoded tissues and sex
        #x = pd.concat([x,pd.get_dummies(data['Tissue'], prefix='tissue'), pd.get_dummies(data['Sex'], prefix='sex')],axis=1)
        
        steps = [('standardization', StandardScaler()), ('normalization', MinMaxScaler())]
        pre_processing_pipeline = Pipeline(steps)
        transformed_data = pre_processing_pipeline.fit_transform(x)

        x = transformed_data
        
        print('Data length', len(x))
        
        return x #normalize(x, axis=0) if normalize_expressions else x
    
    def get_features_dataframe(self, add_tissues=False):
        data = self.samples.join(self.expressions, on = "Name", how="inner")
        ji = data.columns.drop(self.drop_list)
        df = data[ji]
        if add_tissues:
            df = pd.concat([df,pd.get_dummies(data['Tissue'], prefix='tissue'), pd.get_dummies(data['Sex'], prefix='sex')],axis=1)
        x = df.values
        
        min_max_scaler = MinMaxScaler()
        x_scaled = min_max_scaler.fit_transform(x)
        df_normalized = pd.DataFrame(x_scaled, columns=df.columns, index=df.index)
        return df_normalized


In [9]:
samples_path = '../data/gtex/v8_samples.parquet'
expressions_path = '../data/gtex/v8_expressions.parquet'

In [10]:
genes = Genes(samples_path, expressions_path, problem_type="regression")
X = genes.prepare_data(True)
Y = genes.Y

Data length 15343


### Preprocessing for Classification model

In [11]:
from sklearn import model_selection
X_1, X_2, y_1, y_2 = model_selection.train_test_split(X, Y, test_size=0.5, random_state=seed)

In [12]:
_dtype = np.float32
y_1 = np.vstack(y_1).astype(np.float32)
y_2 = np.vstack(y_2).astype(np.float32)
X_1.dtype, y_1.dtype,X_2.dtype, y_2.dtype

(dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'))

In [14]:
sy.version.__version__

'0.2.8'

<h2>Setup config</h2>
Init hook, connect with grid nodes, etc...

In [40]:
hook = sy.TorchHook(torch)

compute_nodes = []
for node in nodes:
    # For syft 0.2.8 --> replace DynamicFLClient with DataCentricFLClient
    compute_nodes.append( DataCentricFLClient(hook, node) )

In [41]:
compute_nodes

[<Federated Worker id:h1>, <Federated Worker id:h2>]

## 1 - Conversion to Tensor

The code below will convert GTEx data samples to tensors.

In [42]:
shared_x1, shared_x2, shared_y1, shared_y2 = X_1, X_2, y_1, y_2

# Convert numpy array to torch tensors -->
shared_x1 = torch.from_numpy(shared_x1)
shared_x2 = torch.from_numpy(shared_x2)
shared_y1 = torch.from_numpy(shared_y1)
shared_y2 = torch.from_numpy(shared_y2)

shared_x1 = torch.tensor(shared_x1, dtype=torch.float32)
shared_x2 = torch.tensor(shared_x2, dtype=torch.float32)
shared_y1 = torch.tensor(shared_y1, dtype=torch.float32)
shared_y2 = torch.tensor(shared_y2, dtype=torch.float32)

datasets  = [shared_x1, shared_x2]
labels = [shared_y1, shared_y2]

/Users/pankajverma/anaconda3/envs/pysyft_v028/lib/python3.7/site-packages/syft/frameworks/torch/hook/hook.py:560: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  current_tensor = hook_self.torch.native_tensor(*args, **kwargs)


In [43]:
shared_x1[0], shared_y2[0]

(tensor([0.0149, 0.0696, 0.2304,  ..., 0.1199, 0.0991, 0.1440]),
 tensor([64.5000]))

In [48]:
# Concatenate 
X = torch.cat((shared_x1, shared_x2), dim=0)
Y = torch.cat((shared_y1, shared_y2), dim=0)
rmse = []
mae = []
r2 = []
huber_loss = []

from torch import nn, optim
import torch.nn.functional as F
import torch

loss_func = torch.nn.MSELoss()

# TODO: Define your network architecture here
class Regression(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.fc1 = nn.Linear(num_features, 1024)
        self.dropout = nn.Dropout(0.1) 
        self.fc2 = nn.Linear(1024, 512)
        self.dropout = nn.Dropout(0.1) 
        self.fc3 = nn.Linear(512, 64)
        self.dropout = nn.Dropout(0.1) 
        self.fc4 = nn.Linear(64, 1)
        
    def forward(self, x):
        # make sure input tensor is flattened
        x = x.view(x.shape[0], -1)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        
        return x
    
# Create the network, define the criterion and optimizer
NUM_FEATURES = 18388
model = Regression(NUM_FEATURES)
# criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
    
epochs = 2

# Below training in a federated way, but without sending data to nodes (full data) --->

In [50]:
def epoch_total_size(data):
    total = 0
    for i in range(len(data)):
        for j in range(len(data[i])):
            total += data[i][j].shape[0]
            
    return total

data = datasets
target = labels
opt = optimizer
N_EPOCS = 20
rmse = []
mae = []
r2 = []
huber_loss = []

def train(epoch):
    model.train()
    epoch_total = epoch_total_size(data)
    current_epoch_size = 0
    for i in range(len(data)):
        correct = 0

        epoch_loss = 0.0
        epoch_acc = 0.0
        current_epoch_size += len(data[i])

        opt.zero_grad()
        pred = model(data[i])
        loss = loss_func(pred, target[i])
        loss.backward()
        opt.step()

        y_pred = pred.detach().numpy()
        y_true = target[i].detach().numpy()

        rmse.append(mean_squared_error(y_true, y_pred))
        mae.append(mean_absolute_error(y_true, y_pred))
        r2.append(r2_score(y_true, y_pred))

        print('Train Epoch: {} | With h{} data |: Train Loss: {:.3f} | R^2: {:.3f} | Mean squared error: {:.3f} | Mean absolute error: {:.3f}'.format(
                  epoch, i+1, loss, r2_score(y_true, y_pred), mean_squared_error(y_true, y_pred), mean_absolute_error(y_true, y_pred)))

for epoch in range(N_EPOCS):
    train(epoch)

Train Epoch: 0 | With h1 data |: Train Loss: 2908.185 | R^2: -16.952 | Mean squared error: 2908.185 | Mean absolute error: 52.405
Train Epoch: 0 | With h2 data |: Train Loss: 2880.106 | R^2: -16.459 | Mean squared error: 2880.107 | Mean absolute error: 52.109
Train Epoch: 1 | With h1 data |: Train Loss: 2805.562 | R^2: -16.318 | Mean squared error: 2805.562 | Mean absolute error: 51.413
Train Epoch: 1 | With h2 data |: Train Loss: 2612.508 | R^2: -14.837 | Mean squared error: 2612.508 | Mean absolute error: 49.456
Train Epoch: 2 | With h1 data |: Train Loss: 2287.957 | R^2: -13.123 | Mean squared error: 2287.957 | Mean absolute error: 46.024
Train Epoch: 2 | With h2 data |: Train Loss: 1801.458 | R^2: -9.921 | Mean squared error: 1801.458 | Mean absolute error: 40.170
Train Epoch: 3 | With h1 data |: Train Loss: 1193.730 | R^2: -6.369 | Mean squared error: 1193.730 | Mean absolute error: 31.374
Train Epoch: 3 | With h2 data |: Train Loss: 631.115 | R^2: -2.826 | Mean squared error: 631

KeyboardInterrupt: 

<h2>2 - Tagging tensors</h2>
The code below will add a tag (of your choice) to the data that will be sent to grid nodes. This tag is important as the network will need it to retrieve this data later.

In [44]:
dummy_data = [datasets[0][0:200], datasets[1][0:200]]
dummy_label = [labels[0][0:200], labels[1][0:200]]

In [45]:
dummy_data

[tensor([[0.0149, 0.0696, 0.2304,  ..., 0.1199, 0.0991, 0.1440],
         [0.0000, 0.0322, 0.1970,  ..., 0.0884, 0.0457, 0.2258],
         [0.0142, 0.0029, 0.1234,  ..., 0.3109, 0.1795, 0.4097],
         ...,
         [0.0352, 0.0050, 0.2004,  ..., 0.2331, 0.1092, 0.3676],
         [0.0825, 0.0218, 0.4244,  ..., 0.1037, 0.0431, 0.2743],
         [0.0142, 0.0307, 0.2735,  ..., 0.1049, 0.0498, 0.1756]]),
 tensor([[0.0085, 0.0005, 0.0511,  ..., 0.0172, 0.0095, 0.0196],
         [0.0271, 0.0364, 0.0961,  ..., 0.3226, 0.1927, 0.4173],
         [0.0474, 0.0038, 0.3967,  ..., 0.3229, 0.2227, 0.1453],
         ...,
         [0.1076, 0.0026, 0.2818,  ..., 0.1928, 0.0770, 0.2906],
         [0.0262, 0.0849, 0.2029,  ..., 0.0505, 0.0225, 0.1307],
         [0.0277, 0.0046, 0.1387,  ..., 0.1874, 0.1068, 0.3855]])]

In [46]:
tag_input = []
tag_label = []

for i in range(len(compute_nodes)):
    tag_input.append(dummy_data[i].tag("#X", "#gtex_v8", "#dataset","#balanced").describe("The input datapoints to the GTEx_V8 dataset."))
    tag_label.append(dummy_label[i].tag("#Y", "#gtex_v8", "#dataset","#balanced").describe("The input labels to the GTEx_V8 dataset."))

<h2> 3 - Sending our tensors to grid nodes</h2>

In [47]:
shared_x1 = tag_input[0].send(compute_nodes[0]) # First chunk of dataset to h1
shared_x2 = tag_input[1].send(compute_nodes[1]) # Second chunk of dataset to h2

shared_y1 = tag_label[0].send(compute_nodes[0]) # First chunk of labels to h1
shared_y2 = tag_label[1].send(compute_nodes[1]) # Second chunk of labels to h2

WebSocketConnectionClosedException: Connection is already closed.

In [27]:
print("X tensor pointers: ", shared_x1, shared_x2)
print("Y tensor pointers: ", shared_y1, shared_y2)

X tensor pointers:  (Wrapper)>[PointerTensor | me:70361333315 -> h1:35881159115]
	Tags: #balanced #dataset #gtex_v8 #X 
	Shape: torch.Size([900, 18420])
	Description: The input datapoints to the GTEx_V8 dataset.... (Wrapper)>[PointerTensor | me:49684275468 -> h2:52723085225]
	Tags: #balanced #dataset #gtex_v8 #X 
	Shape: torch.Size([900, 18420])
	Description: The input datapoints to the GTEx_V8 dataset....
Y tensor pointers:  (Wrapper)>[PointerTensor | me:49137384384 -> h1:4180671457]
	Tags: #balanced #Y #gtex_v8 #dataset 
	Shape: torch.Size([900])
	Description: The input labels to the GTEx_V8 dataset.... (Wrapper)>[PointerTensor | me:10561419581 -> h2:1629165329]
	Tags: #balanced #Y #gtex_v8 #dataset 
	Shape: torch.Size([900])
	Description: The input labels to the GTEx_V8 dataset....


In [27]:
shared_x1[0]

(Wrapper)>[PointerTensor | me:9001755644 -> h1:12992579137]

<h2>Disconnect nodes</h2>

In [28]:
for i in range(len(compute_nodes)):
    compute_nodes[i].close()

### Go to the following address to search available tags:
http://0.0.0.0:5000/search-available-tags